In [1]:
# import modules for Qiskit
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, BasicAer, Aer, IBMQ, execute, schedule
from qiskit.providers.aer.noise import NoiseModel
from qiskit.tools.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from qiskit.providers.aer import AerSimulator
import qiskit.providers.aer.noise as Noise
from qiskit.compiler import transpile
from qiskit.transpiler import PassManager

import numpy as np
np.set_printoptions(precision=3)
from  IPython.core.display import display

# import modules for SHA
import sys
import hashlib
import numpy as np
import pandas as pd
import random

if sys.version_info < (3, 6):
	import sha3

# import other useful packages 
from numpy import pi
from datetime import date
import time
import os
import seaborn as sns
import matplotlib.pyplot as plt
import math
import time

# code for getting the backend

print("Getting provider...")
if not IBMQ.active_account():
    IBMQ.load_account()
provider = IBMQ.get_provider()

global backend, noise_model

def build_noise_model(backend):
    """
    build_noise_model builds a noise model based on the backend
    
    :param backend: the backend that is used to build the noise model
    :return: the noise model for noise simulation
    """

    noise_model = NoiseModel.from_backend(backend)
    return(noise_model)

# parameters needed to run a noise model
# if using a real quantum computer, only the backend is used
backend = 'ibm_nairobi'
noise_model = build_noise_model(backend)
basis_gates = noise_model.basis_gates
coupling_map = backend.configuration().coupling_map

folder='Results'
if(not os.path.isdir(str(folder))):
    os.makedirs(str(folder))
filepath = os.getcwd()+'/'+str(folder)+'/' #specify data storage folder
    
# settings for IBM Q backends
# TOKEN = ''
# IBMQ.save_account(TOKEN)
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
simulator_name = 'qasm_simulator'

# parameters
numberOfcalcs = 100
MAX_NONCE = 2**32

# input parameters
previous_hash = '' # empty hash
block_number = 0
transactions ='Schroedinger paid Einstein 1 qBTC'
n_qreg_list = [7] # list of the numbers of qubits to experiment with
num_states = 2620
num_intersection = 100

# list of errors for each number of qubits
error_list = [0]*len(n_qreg_list)


# QPoW cycle: function takes the input text (block number, nonce,...) and pushes it through once 

def qPoW(text, quantum_circuit, errors, num_states, num_intersection):
    """
    qPoW takes the input text and pushes it through once, and it updates the array of errors

    :param text: the text that is hashed and used to parameterize the quantum circuit
    :param quantum_circuit: the function to build the parameterized quantum circuit
    :param errors: the array of errors for different percentages of maxstates and overlap requirements
    :param num_states: the maximum number of states considered
    :param num_intersection: the number of overlap percentages considered (from 0% to 100%)
    :return: the updated error array
    """
    hashIn = hashlib.sha3_256(text.encode("ascii")).hexdigest() # hashing the 'text' input
    #string-type output

    print ('hashIn-hex:', hashIn, 'length:', len(hashIn))

    # convert hashIn(hex) to hashIn_bin(binary)
    scale = 16 # hex base
    hashIn_bin = bin(int(hashIn, scale))[2:].zfill(len(hashIn)*4)
    print ('hashIn-binary:', str(hashIn_bin), 'length:', len(hashIn_bin))

    # comparison of the results produced by quantum simulator and quantum computer
    qstate_bin_sim = sim_quantum_operation(quantum_circuit, hashIn_bin, num_states)
    qstate_bin_exp = exp_quantum_operation(quantum_circuit, hashIn_bin, num_states) #qstate is a 256 binary number

    # error rate calculation
    for states in range(1, num_states+1):
        maxstates_sim = set(qstate_bin_sim[:states])
        maxstates_exp = set(qstate_bin_exp[:states])
        overlap_val = math.ceil(overlap(maxstates_sim, maxstates_exp)*num_intersection/states+1e-20)
        errors[states][overlap_val:num_intersection+1]+=1
    return errors

def overlap(maxstates_sim, maxstates_exp):
    """
    overlap calculates the overlap between two sets

    :param maxstates_sim: the set of states from the simulation
    :param maxstates_exp: the set of states from the noise simulator or quantum computer
    :return: the number of overlapping states
    """
    return len(maxstates_sim.intersection(maxstates_exp))
    
# converting hashIn_bin to a bit string to pass thru a quantum processor
def break_up_4bit_values(hashIn_bin):
    """
    break_up_4bit_values converts the input into an array of 4-bit strings

    :param hashIn_bin: the binary string that is broken up
    :return: array of 4-bit strings
    """
    array_4_bit_values = []

    for i in range(64): 
      four_bits = hashIn_bin[2+4*i:2+4*i+4]
      array_4_bit_values.append(four_bits)
        
    print("hashIn binary split into 4bit bins:", array_4_bit_values)
    return array_4_bit_values

def quantum_circuit(q_par, n_qreg, circ_layer = 1):
    """
    quantum_circuit builds a paramterized quantum circuit

    :param q_par: the parameters
    :param n_qreg: the number of qubits
    :param circ_layer: the number of times to repeat the circuit
    :return: the quantum circuit
    """
    k = 0 # counter for the parameter values
    
    # circuit 15
    # setting the quantum circuit:
    qreg_q = QuantumRegister(n_qreg, 'q')
    creg_c = ClassicalRegister(n_qreg, 'c')
    circuit = QuantumCircuit(qreg_q, creg_c)
    for layer in range(circ_layer):

        for i in range(n_qreg):   
            circuit.ry(pi/2, qreg_q[i])

        for i in range(0, n_qreg):
            circuit.crx(q_par[k+i]*pi/8, qreg_q[n_qreg-i-1], qreg_q[(n_qreg-i)%n_qreg])
        k+=n_qreg
        
        for i in range(n_qreg):
            circuit.ry(pi/2, qreg_q[i])
        
        for i in range(0, n_qreg):
            circuit.crx(q_par[k+i]*pi/8, qreg_q[(n_qreg-1+i)%n_qreg], qreg_q[(n_qreg-2+i)%n_qreg])
        k+=n_qreg

    # measurements of all qubits
    for i in range(n_qreg):
        circuit.measure(qreg_q[i], creg_c[i])

    return circuit


def exp_quantum_operation(quantum_circuit, hashIn, num_states, use_noise_model = False):
    """
    exp_quantum_operation runs the paramterized circuit with either a noise simulator or a real quantum computer

    :param quantum_circuit: function that returns the desired quantum circuit
    :param hashIn: the SHA3-256 hash of the input string
    :param num_states: the maximum number of states considered
    :param use_noise_model: whether to use the noise model or the real quantum computer
    :return: the top num_states output states in an array
    """
    # input hashIn string
    fourbit_array = break_up_4bit_values(hashIn)
    q_par = [int(fourbit_array[i],2) for i in range(len(fourbit_array)-1)] #throwing away the last string element
    circuit = quantum_circuit(q_par, n_qreg)
    
    if(use_noise_model):
        ######## Real quantum computer runs ########
        # noise model not used
        transpiled_circuit = transpile(circuit, backend, seed_transpiler=13)
        qpu_job = backend.run(transpiled_circuit, shots=20000)
        job_id = qpu_job.job_id()
        job_monitor(qpu_job)
        results = qpu_job.result()
    else:
        ######## Noise simulator runs ########
        # noise model used
        backend = Aer.get_backend(simulator_name)
        job = execute(circuit, backend, noise_model=noise_model, coupling_map=coupling_map, basis_gates=basis_gates)
        job_monitor(job)
        results = job.result()

    counts = results.get_counts(circuit)

    # picking up the maximally probable states
    max_state256 = sorted(counts, key=counts.get, reverse=True)[:num_states]
    
    return max_state256 # 4bit vector

# quantum simulator run
def sim_quantum_operation(quantum_circuit, hashIn, num_states):
    """
    sim_quantum_operation runs the paramterized circuit with the qasm simulator

    :param quantum_circuit: function that returns the desired quantum circuit
    :param hashIn: the SHA3-256 hash of the input string
    :param num_states: the maximum number of states considered
    :return: the top num_states output states in an array
    """
    # input hashIn string
    fourbit_array = break_up_4bit_values(hashIn)
    q_par = [int(fourbit_array[i],2) for i in range(len(fourbit_array)-1)] #throwing away the last string element
    circuit = quantum_circuit(q_par, n_qreg)

    backend = BasicAer.get_backend(simulator_name) # run on local simulator by default 

    job = execute(circuit, backend, shots=20000)

    # Monitor job progress and wait until complete:
    job_monitor(job)

    # Get the job results (this method also waits for the job to complete):
    results = job.result()
    
    counts = results.get_counts(circuit)

    # picking up the maximally probable states
    max_state256 = sorted(counts, key=counts.get, reverse=True)[:num_states]

    return max_state256 # 4bit vector

for i in range(len(n_qreg_list)):
    n_qreg = n_qreg_list[i]
    counter = 1
    errors = np.zeros((num_states+1, num_intersection+1)) # number of 'False' nonces
    while counter <= numberOfcalcs:

        ### Execute both exp and qPoW
        nonce = random.randint(0, MAX_NONCE)
        print('nonce:', nonce, '\n') 
        text = str(block_number) + transactions + previous_hash + str(nonce) #hash input
        print('text:', text, '\n')

        errors = qPoW(text, quantum_circuit, errors, num_states, num_intersection)
        error_rate = np.array(errors)/counter

        print('error rate: ' + str(error_rate))
        print(counter)
        counter+=1
    error_list[i] = error_rate
for idx, x in enumerate(error_list):
    print(x)
    pd.DataFrame(x).to_csv(os.path.join(folder, f'{n_qreg_list[idx]}qubit_errors.csv'))

ModuleNotFoundError: No module named 'qiskit'